# Phylogenetic trees

We describe how the decomposition method works on a trilinear tensor of $\mathbb{R}^4 \otimes \mathbb{R}^4 \otimes \mathbb{R}^4$ of rank $4$:

## Problem
We are given a tensor $T$ of the form 
$$
T = \sum_{i=1}^{4} \omega_i A[:,i] \otimes B[:,i] \otimes C[:,i]
$$
where $A,B,C \in \mathbb{R}^{4 \times 4}$ are Markow matrices and $\omega=(\omega_1, \ldots, \omega_4)\in (\mathbb{R}_+)^{4}$ are positive weights.


**How to recover the weights $\omega$ and the factors $A,B,C$ from the coefficients of $T$.**

In [1]:
using TensorDec, DynamicPolynomials, LinearAlgebra

# scale the columns of A, B, C and the weights w so that the sum of the colmuns is 1
normalize_markov! = function(w,A,B,C) 
    for i in 1:size(A,2) 
        l = sum(A[j,i] for j in 1:size(A,1) ) 
        A[:,i] /= l
        w[i] *= l
    end
    for i in 1:size(B,2) 
        l = sum(B[j,i] for j in 1:size(B,1) ) 
        B[:,i] /=l
        w[i] *= l
    end
    for i in 1:size(C,2) 
        l = sum(C[j,i] for j in 1:size(B,1) ) 
        C[:,i] /=l
        w[i] *= l
    end
    w, A,B,C
end;
# scale the columns of A so that the first row is [1, ..., 1]
normalize_affine = function(A0::AbstractMatrix) A = Matrix(A0); for i in 1:size(A,2)  A[:,i] /= A[1,i] end; A end;

In [2]:
A = rand(4,4); B = rand(4,4); C = rand(4,4); w = rand(4);
normalize_markov!(w,A,B,C);

We verify that the columns sum to `1.`:

In [3]:
fill(1.,4)'*A, fill(1.,4)'*B, fill(1.,4)'*C

([1.0 0.9999999999999999 1.0 0.9999999999999999], [1.0 1.0 1.0 0.9999999999999999], [1.0 1.0 0.9999999999999998 1.0])

In [4]:
#w = fill(1.,4); A=[1 1 1 1; 0 1. 0 0; 0 0 2. 0; 0 0 0 3.]; B= A; C=A

In [5]:
T = tensor(w,A,B,C)

4×4×4 Array{Float64, 3}:
[:, :, 1] =
 0.63856   0.498562  0.609002  0.492611
 0.659005  0.576221  0.588409  0.480191
 0.331819  0.218192  0.31365   0.291366
 0.217415  0.142356  0.211895  0.192002

[:, :, 2] =
 0.459046  0.304443  0.516376  0.307711
 0.31125   0.259229  0.270975  0.217506
 0.323017  0.194411  0.324703  0.241424
 0.20303   0.116249  0.226682  0.158003

[:, :, 3] =
 0.147284   0.102441   0.122958   0.0951209
 0.0896452  0.0735928  0.0659168  0.0550932
 0.129666   0.0858343  0.0949834  0.0865085
 0.0728417  0.04477    0.0620078  0.0559637

[:, :, 4] =
 0.320664  0.218559   0.39174   0.197574
 0.240617  0.205661   0.21549   0.166941
 0.191538  0.117397   0.215483  0.130113
 0.124318  0.0727524  0.155088  0.0859419

The corresponding polynomial is:

In [6]:
X = @polyvar x0 x1 x2 x3; Y = @polyvar y0 y1 y2 y3; Z = @polyvar z0 z1 z2 z3;
F= sum(T[i,j,k]*X[i]*Y[j]*Z[k] for i in 1:4 for j=1:4 for k=1:4)

0.0859418985158878x3y3z3 + 0.05596366474953316x3y3z2 + 0.1580025087257289x3y3z1 + 0.19200166860542914x3y3z0 + 0.15508777176200633x3y2z3 + 0.062007785735641446x3y2z2 + 0.226681759454738x3y2z1 + 0.21189505440504092x3y2z0 + 0.07275241058971704x3y1z3 + 0.04477003313863812x3y1z2 + 0.11624937787793596x3y1z1 + 0.14235563729401707x3y1z0 + 0.12431821065686982x3y0z3 + 0.07284167351701111x3y0z2 + 0.2030304440318155x3y0z1 + 0.21741506073333636x3y0z0 + 0.1301133623512499x2y3z3 + 0.08650854000696753x2y3z2 + 0.24142382069130386x2y3z1 + 0.2913663638090841x2y3z0 + 0.21548273047272626x2y2z3 + 0.09498339161454605x2y2z2 + 0.3247027683869597x2y2z1 + 0.31365015226504855x2y2z0 + 0.1173969630002421x2y1z3 + 0.08583432329201908x2y1z2 + 0.19441116803587513x2y1z1 + 0.21819242797364746x2y1z0 + 0.19153836644250666x2y0z3 + 0.12966581933685403x2y0z2 + 0.32301710108580384x2y0z1 + 0.33181877113093455x2y0z0 + 0.16694114767271573x1y3z3 + 0.0550931886254093x1y3z2 + 0.21750590977692524x1y3z1 + 0.4801909253352575x1y3z0 + 0.

In [7]:
H = [T[:,:,i] for i in 1:4];

## Matrices from the tensor
The flattening or Hankel matrix of the tensor indexed by monomials $A=(x_i)_{0\le i\le 3}$, $A'=(y_j\, z_k)_{0 \le j,k\le 3}$ is: 

In [8]:
hcat(H...)

4×16 Matrix{Float64}:
 0.63856   0.498562  0.609002  0.492611  …  0.218559   0.39174   0.197574
 0.659005  0.576221  0.588409  0.480191     0.205661   0.21549   0.166941
 0.331819  0.218192  0.31365   0.291366     0.117397   0.215483  0.130113
 0.217415  0.142356  0.211895  0.192002     0.0727524  0.155088  0.0859419

By scaling the factors and the weights of the decomposition, we assume that the factors $U,V,W$ have their first coordinates equal to $1$. We set $x_0=1$, $y_0=1$, $z_0=1$ and work on an affine chart of $\mathbb{P}^3\times \mathbb{P}^3\times\mathbb{P}^3$.

A basis of $\mathcal{A}$ is $B=\{1, y_1, y_2,y_3\}$. The operators $M_i$ of multiplication by ${z_i}$ in the basis $B$ of $\mathcal{A}$ are:

In [9]:
M = [inv(H[1])*H[i] for i in 2:4];

## Joint diagonalisation

We take a random combination $M_{rnd}$ of $M_i$ and compute its eigenvectors:

In [10]:
Mrnd = sum(M[i]*rand() for i in 1:3);

In [11]:
E = eigen(Mrnd).vectors

4×4 Matrix{Float64}:
 -0.677549   0.671611  -0.399447   0.818269
  0.69611   -0.69861    0.510161  -0.373124
  0.161483  -0.178923  -0.515211  -0.256734
  0.174016   0.169907   0.561013  -0.353981

We verify that the operators of multiplication $M_i$ are diagonal (up to numerical error) in the basis of eigenvectors of $M_{rnd}$:

In [12]:
D = [inv(E)*M[i]*E  for i in 1:3];

In [13]:
D[1]

4×4 Matrix{Float64}:
  0.384224      1.22402e-14  -4.45199e-14  -1.49325e-14
 -7.77156e-16   1.00375      -3.35287e-14  -1.22125e-14
 -6.73073e-15   3.69843e-15   3.18572       7.68829e-15
  9.4022e-15   -8.44463e-15   1.27676e-15   3.56338

In [14]:
D[2]

4×4 Matrix{Float64}:
  0.0804996    7.34135e-15  -3.58047e-15  9.50628e-15
 -6.43929e-15  0.389043      2.88658e-15  5.21805e-15
  3.08781e-16  9.09862e-16   0.434225     2.05391e-15
 -6.7793e-15   5.46785e-15  -1.44051e-14  2.95996

In [15]:
D[3]

4×4 Matrix{Float64}:
  0.330232      1.78468e-14  -1.29896e-14   6.10623e-15
 -8.60423e-15   0.472936     -1.9762e-14    4.21885e-15
  4.4964e-15   -6.50868e-15   2.82149      -6.18949e-15
  6.83481e-15  -3.38271e-15   3.11001e-14   1.79256

## The factors from the eigenvectors

The corresponding terms on the diagonal give the factor $W$ (with the first coordinate equal to $1$):

In [16]:
W = fill(1.,4,4); for i in 1:3 for j in 1:4 W[i+1,j] = D[i][j,j] end end; W

4×4 Matrix{Float64}:
 1.0        1.0       1.0       1.0
 0.384224   1.00375   3.18572   3.56338
 0.0804996  0.389043  0.434225  2.95996
 0.330232   0.472936  2.82149   1.79256

The eigenvectors `E` are (up to a scalar) the interpolation polynomials at the points in the basis $B$. 
Therefore, `inv(E)'` is (up to a scaling of the columns) the **Vandermonde** matrix of the points in $B=\{1, y_1, y_2, y_3\}$:

$$
V = \left( 
\begin{array}{cccc}
1 & 1 & 1 & 1 \\
v_{1,1} & v_{2,1} & v_{3,1} & v_{4,1} \\
v_{1,2} & v_{2,2} & v_{3,2} & v_{4,2} \\
v_{1,3} & v_{2,3} & v_{3,3} & v_{4,3} \\
\end{array}
\right)
$$
This gives the factor `V`

In [17]:
V = inv(E)'

4×4 adjoint(::Matrix{Float64}) with eltype Float64:
 4.54037  2.71845  -0.717092  2.40036
 4.10939  1.28546  -0.264018  2.21873
 4.05693  2.53245  -1.75912   0.421948
 3.2216   3.09232  -0.103501  0.0789611

We remove the scaling factors, by normalizing the columns so that the first coordinate of the points is $1$:

In [18]:
V = normalize_affine(V)

4×4 Matrix{Float64}:
 1.0       1.0       1.0       1.0
 0.905077  0.472864  0.368179  0.924333
 0.893524  0.93158   2.45313   0.175785
 0.709546  1.13753   0.144335  0.0328955

In [19]:
U = normalize_affine(H[1]*E)

4×4 Matrix{Float64}:
 1.0       1.0       1.0        1.0
 1.35261   0.29567   0.0795717  0.555197
 0.288573  1.15391   0.507942   1.11787
 0.197175  0.746456  0.411169   0.420357

## The weights from the factors

We solve a Vandermonde system 
$$
\left( 
\begin{array}{cccc}
\vdots & &  &\vdots \\
u_{1,i}v_{1,j}w_{1,k} & & & u_{4,i}v_{4,j} w_{4,k} \\
\vdots & & & \vdots \\
\end{array}
\right) \, 
\left( 
\begin{array}{c}
\omega_1\\ \vdots \\ \omega_4 \end{array}\right)= 
\left( 
\begin{array}{c}
\vdots \\
T[i,j,k]\\
\vdots
\end{array}
\right)
$$
selecting only the rows where $k=1$, s.t. $w_{l,1}=1$ just to simplify the matrix constructions.

In [20]:
Vdm = hcat([[U[i,k]*V[j,k] for i in 1:4 for j in 1:4] for k in 1:4]...)
theta = [H[1][i,j] for i in 1:4 for j in 1:4];

w1 = Vdm\theta

4-element Vector{Float64}:
 0.4470649786874177
 0.15032225861369847
 0.027340579174909594
 0.013831789444849434

In [21]:
normalize_markov!(w1,U,V,W); T1 = tensor(w1,U,V,W); norm(T-T1)

5.750349421984536e-14